In [1]:
import os
import numpy as np
import cv2

# 이미지가 저장된 디렉터리 경로
image_directory = "./image/images"

# 객체 감지된 결과를 읽어올 파일 경로
results_directory = "./image/labels"

# 이미지 파일 이름이 "image1.jpg", "image2.jpg", ..., "image60.jpg"와 같은 형식이라고 가정
for i in range(1, 343):
    image_path = os.path.join(image_directory,f"{i}.jpg")
    results_path = os.path.join(results_directory,f"{i}.txt")
    
    # 이미지 불러오기
    image = cv2.imread(image_path)

    # 결과 파일 읽어오기
    with open(results_path, "r") as file:
        lines = file.readlines()

    # 바운딩 박스 정보를 저장할 리스트 초기화
    bounding_boxes = []

    # 결과를 이미지에 그리기
    for line in lines:
        line = line.strip().split(" ")
        class_id = int(line[0])
        confidence = float(line[1])
        x, y, w, h = map(float, line[1:])  # 클래스 ID와 신뢰도를 제외한 바운딩 박스 좌표만 사용

        # 바운딩 박스 좌표 계산
        left = int((x - w / 2) * image.shape[1])
        top = int((y - h / 2) * image.shape[0])
        right = int((x + w / 2) * image.shape[1])
        bottom = int((y + h / 2) * image.shape[0])

        # 바운딩 박스 정보를 저장
        bounding_boxes.append((left, top, right, bottom, confidence, class_id))

    # 새로운 이미지 생성
    output_image = np.copy(image)

    # 객체 이미지를 저장할 폴더 생성
    output_folder = f"./detected_images"
    os.makedirs(output_folder, exist_ok=True)

    # 각 클래스에 따라 디렉토리 생성 및 이미지 저장
    for bbox in bounding_boxes:
        left, top, right, bottom, confidence, class_id = bbox
        class_folder = f"{output_folder}/class_{class_id}/"
        os.makedirs(class_folder, exist_ok=True)

        # 바운딩 박스 그리기
        cv2.rectangle(output_image, (int(left), int(top)), (int(right), int(bottom)), (0, 255, 0), 2)
        # 객체가 감지된 부분만 이미지로 저장
        object_image = image[int(top):int(bottom), int(left):int(right)]
        # 이미지 저장
        cv2.imwrite(f"{class_folder}/detected_image_{class_id}_{i}_{confidence:.2f}.jpg", object_image)

    # 새로운 이미지 저장
    output_folder = f"./classified_images/image{i}/"
    os.makedirs(output_folder, exist_ok=True)
    cv2.imwrite(f"{output_folder}/classified_image.jpg", output_image)


Note: you may need to restart the kernel to use updated packages.
